In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
groups = {
    "Peek": {"gene_sets": SETTING["gene_sets_to_peek"], "size": 12, "color": "#20d9ba"}
}

In [ ]:
def plot_hill(score_p_value, groups, name):

    score_p_value.sort_values("Score", inplace=True)

    xs = [1 - score_p_value["P-Value"]]

    ys = [score_p_value["Score"]]

    names = [name]

    markers = [dict(color="#d0d0d0", size=3.2)]

    texts = [score_p_value.index]

    for group_name, group in groups.items():

        group_gene_sets = group["gene_sets"]

        score_p_value__peek = score_p_value.loc[
            [gene_set in group_gene_sets for gene_set in score_p_value.index]
        ]

        xs.append(1 - score_p_value__peek["P-Value"])

        ys.append(score_p_value__peek["Score"])

        names.append(group_name)

        texts.append(score_p_value__peek.index)

        markers.append(dict(size=group["size"], color=group["color"]))

    ccal.plot_points(
        xs,
        ys,
        names=names,
        modes=("lines",) + ("markers",) * len(names),
        texts=texts,
        markers=markers,
        title="{}<br>{}".format(target_name, name),
        xaxis_title="1 - P-Value",
        yaxis_title="Gene Set Score",
        html_file_path="{}/{}".format(output_directory_path, name),
    )

In [ ]:
for target_name in target_x_sample.index:

    output_directory_path = "{}/{}".format(
        PATH["compare_differentially_expressed_gene_set/"], target_name
    )

    ccal.establish_path(output_directory_path, "directory")

    score_moe_p_value_fdr = pd.read_csv(
        "{}/{}/all.tsv".format(
            PATH["find_differentially_expressed_gene_set/"], target_name
        ),
        sep="\t",
        index_col=0,
    )

    plot_hill(score_moe_p_value_fdr, groups, "All")

    for gene_set_file_path in SETTING["gene_set_file_paths"]:

        plot_hill(
            score_moe_p_value_fdr.reindex(ccal.read_gmt(gene_set_file_path).index),
            groups,
            gene_set_file_path.split("/")[-1],
        )